# Webscrapeo : Ubica tu casilla

### Paquetes

In [1]:
import requests
import numpy as np
import pandas as pd

### Pipeline

In [60]:
API = "https://us-central1-ine-ubica-tu-casilla.cloudfunctions.net/utc-api/search/rasgo-with-cmr-uts?e=1&s=488"
headers = {"authority": "us-central1-ine-ubica-tu-casilla.cloudfunctions.net",
            "method": "GET",
            "scheme": "https",
            "accept": "application/json, text/plain, */*",
            "accept-encoding": "gzip, deflate, br",
            "origin": "https://ubicatucasilla.ine.mx",
            "referer": "https://ubicatucasilla.ine.mx/",
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36",
            "x-utc-service-id": "ubicatucasilla",
            "x-utc-version-id": "2022"}

In [84]:
def get_raw_df(API, num_casilla):
    """
    Extrae el dataframe de una casilla a partir del API
     
    Parameters
    API: string
        URL del API
    num_casilla: int
        Numero de casilla a extraer
    """

    new_API = API.replace("s=488", f"s={num_casilla}")
    r = requests.get(new_API, headers=headers)

    if r.json()['status'] == False:
        raw_df = pd.DataFrame([])
        content = {}
    else:
        content = r.json()['data'][0]['items']
        
        raw_df = pd.DataFrame(content[0]['apellidos'])
        raw_df['domicilio'] = content[0]['data']['domicilio']
        raw_df['ubicacion'] = content[0]['data']['ubicacion']

        raw_df['entidad'] = content[0]['entidadSede']
        raw_df['distitoSede'] = content[0]['distritoSede']

        raw_df 

    return raw_df, content

In [83]:
raw_df, content = get_raw_df(API, 488)
content

[{'code': '03',
  'seccionSede': 430,
  'entidadSede': 1,
  'distritoSede': 1,
  'distrito_localSede': 3,
  'utSede': 13,
  'secciones': ['0430', '0448', '0451', '0488'],
  'apellidos': [{'casilla': 'B',
    'inicial': 'ACEVEDO ESPARZA',
    'final': 'IBARRA GUTIERREZ'},
   {'casilla': 'C1', 'inicial': 'IBARRA GUTIERREZ', 'final': 'ZUBIA MOLINA'}],
  'reference': {'dce': {'geom': {'coordinates': [-102.336903477, 22.121603974],
     'type': 'Point'},
    'ref': {'distrito_federal': 1,
     'distrito_local': 3,
     'entidad': 1,
     'seccion': 430},
    'version': '2021'}},
  'data': {'ut': 13,
   'active': True,
   'domicilio': 'CALLE 5 DE MAYO, NÚMERO 114, SANTIAGO, 20667, PABELLÓN DE ARTEAGA, AGUASCALIENTES',
   'referencia': 'ENTRE 5 DE MAYO Y EMILIANO ZAPATA',
   'ubicacion': 'ESCUELA PRIMARIA 16 DE SEPTIEMBRE',
   'dce': {'coordinates': [-102.336903477, 22.121603974], 'type': 'Point'},
   'geom': {'coordinates': [-102.336903477, 22.121603974], 'type': 'Point'}}}]

In [ ]:
def get_clean_df(raw_df, content):
    """
    """
    raw_secciones = content[0]['secciones']

    clean_secciones = [int(x) for x in raw_secciones]
    complete_df = pd.DataFrame([])

    for clean_seccion in clean_secciones:
        copy_df = raw_df.copy()
        copy_df['seccion'] = clean_seccion

        complete_df = pd.concat([complete_df, copy_df], 
                                ignore_index=True)

In [80]:

raw_df

,casilla,inicial,final,domicilio,ubicacion,entidad,distitoSede
0,B,ACEVEDO ESPARZA,IBARRA GUTIERREZ,"CALLE 5 DE MAYO, NÚMERO 114, SANTIAGO, 20667, ...",ESCUELA PRIMARIA 16 DE SEPTIEMBRE,1,1
1,C1,IBARRA GUTIERREZ,ZUBIA MOLINA,"CALLE 5 DE MAYO, NÚMERO 114, SANTIAGO, 20667, ...",ESCUELA PRIMARIA 16 DE SEPTIEMBRE,1,1


In [68]:
complete_df

,casilla,inicial,final,domicilio,ubicacion,entidad,distitoSede,seccion
0,B,ACEVEDO ESPARZA,IBARRA GUTIERREZ,"CALLE 5 DE MAYO, NÚMERO 114, SANTIAGO, 20667, ...",ESCUELA PRIMARIA 16 DE SEPTIEMBRE,1,1,430
1,C1,IBARRA GUTIERREZ,ZUBIA MOLINA,"CALLE 5 DE MAYO, NÚMERO 114, SANTIAGO, 20667, ...",ESCUELA PRIMARIA 16 DE SEPTIEMBRE,1,1,430
2,B,ACEVEDO ESPARZA,IBARRA GUTIERREZ,"CALLE 5 DE MAYO, NÚMERO 114, SANTIAGO, 20667, ...",ESCUELA PRIMARIA 16 DE SEPTIEMBRE,1,1,448
3,C1,IBARRA GUTIERREZ,ZUBIA MOLINA,"CALLE 5 DE MAYO, NÚMERO 114, SANTIAGO, 20667, ...",ESCUELA PRIMARIA 16 DE SEPTIEMBRE,1,1,448
4,B,ACEVEDO ESPARZA,IBARRA GUTIERREZ,"CALLE 5 DE MAYO, NÚMERO 114, SANTIAGO, 20667, ...",ESCUELA PRIMARIA 16 DE SEPTIEMBRE,1,1,451
5,C1,IBARRA GUTIERREZ,ZUBIA MOLINA,"CALLE 5 DE MAYO, NÚMERO 114, SANTIAGO, 20667, ...",ESCUELA PRIMARIA 16 DE SEPTIEMBRE,1,1,451
6,B,ACEVEDO ESPARZA,IBARRA GUTIERREZ,"CALLE 5 DE MAYO, NÚMERO 114, SANTIAGO, 20667, ...",ESCUELA PRIMARIA 16 DE SEPTIEMBRE,1,1,488
7,C1,IBARRA GUTIERREZ,ZUBIA MOLINA,"CALLE 5 DE MAYO, NÚMERO 114, SANTIAGO, 20667, ...",ESCUELA PRIMARIA 16 DE SEPTIEMBRE,1,1,488


In [35]:
content[0]['secciones']

['0489', '0499', '0500', '0501', '0504', '0505', '0506', '0507']

In [37]:
secciones =  content[0]['secciones']
secciones

['0489', '0499', '0500', '0501', '0504', '0505', '0506', '0507']